# Key-words screening

In [2]:
df = pd.read_csv('datasets/cleaned-datasets/socialisolation_cleaned.csv')

In [8]:
documents_list = []
for text in df_out['text'].values:
    doc = text.split(' ')
    documents_list.append(doc)
    
bi_gram = Phrases(documents_list)

bi_gram_counter = Counter()
for key in bi_gram.vocab.keys():
    bi_gram_counter[key] += bi_gram.vocab[key]
 
for key, counts in bi_gram_counter.most_common(1000):
    print('{0: <30} {1}'.format(str(key), counts))

b'social'                      1136
b'older'                       976
b'peopl'                       612
b'age'                         564
b'use'                         547
b'health'                      502
b'commun'                      500
b'adult'                       490
b'studi'                       484
b'loneli'                      447
b'older_adult'                 422
b'elderli'                     384
b'isol'                        378
b'care'                        352
b'life'                        342
b'live'                        330
b'activ'                       308
b'older_peopl'                 302
b'depress'                     279
b'support'                     278
b'particip'                    270
b'group'                       260
b'social_isol'                 252
b'increas'                     231
b'need'                        231
b'associ'                      228
b'result'                      220
b'factor'                      215
b'technolog'       

In [3]:
df_out = df[df['is_in'] == 0]

In [4]:
df_out.head()

,text,is_intervention,is_in
0,therapeut effect indoor garden programm older ...,1,0
1,depress modifi factor fear older faller transi...,1,0
2,factori structur measur invari pana spanish ol...,1,0
3,experienti variabl influenc play foot-clasp mo...,1,0
4,longev increas posit self-percept age research...,1,0


In [1]:
import nltk
from nltk.stem.porter import PorterStemmer
import csv
import string
from gensim.models import Phrases
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

from collections import Counter

# 2-grams

# 3-grams

In [9]:
three_gram = Phrases(bi_gram[documents_list])

three_gram_counter = Counter()
for key in three_gram.vocab.keys():
    three_gram_counter[key] += three_gram.vocab[key]
 
for key, counts in three_gram_counter.most_common(1000):
    print('{0: <30} {1}'.format(str(key), counts))

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


b'social'                      489
b'use'                         463
b'commun'                      445
b'age'                         419
b'loneli'                      393
b'older_adult'                 371
b'studi'                       340
b'health'                      315
b'elderli'                     292
b'older_peopl'                 281
b'care'                        277
b'particip'                    270
b'social_isol'                 252
b'activ'                       250
b'older'                       234
b'need'                        231
b'depress'                     229
b'group'                       223
b'live'                        218
b'peopl'                       209
b'increas'                     204
b'associ'                      202
b'support'                     196
b'result'                      194
b'develop'                     189
b'may'                         175
b'design'                      174
b'effect'                      171
b'also'             

b'ethic'                       12
b'half'                        12
b'scienc'                      12
b'age_group'                   12
b'sed_load'                    12
b'detail'                      11
b'depress_loneli'              11
b'p_0.001'                     11
b'odd'                         11
b'highest'                     11
b'advantag'                    11
b'today'                       11
b'theoret'                     11
b'sign'                        11
b'certain'                     11
b'principl'                    11
b'action'                      11
b'feedback'                    11
b'speech'                      11
b'hear_aid'                    11
b'combat'                      11
b'analys_use'                  11
b'daili'                       11
b'final'                       11
b'northern'                    11
b'sector'                      11
b'elderli_popul'               11
b'central'                     11
b'assist_technolog'            11
b'capabl'     

# 4-grams

In [10]:
four_gram = Phrases(three_gram[documents_list])

four_gram_counter = Counter()
for key in four_gram.vocab.keys():
    four_gram_counter[key] += four_gram.vocab[key]
 
for key, counts in four_gram_counter.most_common(1000):
    print('{0: <30} {1}'.format(str(key), counts))

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


b'use'                         450
b'commun'                      419
b'age'                         417
b'loneli'                      399
b'older_adult'                 375
b'social'                      344
b'studi'                       317
b'older_peopl'                 281
b'elderli'                     280
b'health'                      264
b'social_isol'                 252
b'care'                        239
b'particip'                    234
b'need'                        231
b'group'                       223
b'depress'                     221
b'activ'                       217
b'increas'                     204
b'associ'                      202
b'peopl'                       195
b'result'                      194
b'live'                        191
b'develop'                     189
b'support'                     187
b'may'                         175
b'design'                      174
b'effect'                      171
b'also'                        171
b'research'         

b'geriatr_depress'             12
b'social_isol_loneli'          12
b'art'                         12
b'music'                       12
b'gfi'                         12
b'epidem'                      12
b'agent'                       12
b'ethic'                       12
b'half'                        12
b'scienc'                      12
b'age_group'                   12
b'sed_load'                    12
b'detail'                      11
b'depress_loneli'              11
b'p_0.001'                     11
b'odd'                         11
b'highest'                     11
b'advantag'                    11
b'today'                       11
b'theoret'                     11
b'sign'                        11
b'certain'                     11
b'principl'                    11
b'action'                      11
b'feedback'                    11
b'speech'                      11
b'hear_aid'                    11
b'combat'                      11
b'analys_use'                  11
b'daili'      

## Find most common key-words

In [35]:
from nltk.text import TextCollection

documents_collection = TextCollection(df_out['text'].values)

vocabulary = []
for document in df_out['text'].values:
    document = list(bi_gram[document.split(' ')])
    vocabulary += document
vocabulary = list(set(vocabulary))

idf_vocab = {}
for word in vocabulary:
    idf_vocab[word] = documents_collection.idf(word)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [36]:
s = sorted(idf_vocab.items(), key=lambda x: x[1])

In [37]:
s

[('lower_level', 0.0),
 ('95_ci', 0.0),
 ('share_space', 0.0),
 ('somat_diseas', 0.0),
 ('intermedi_care', 0.0),
 ('among_elderli', 0.0),
 ('pilot_studi', 0.0),
 ('health_promot', 0.0),
 ('logist_regress', 0.0),
 ('increas_risk', 0.0),
 ('improv_qualiti', 0.0),
 ('experiment_group', 0.0),
 ('use_eas', 0.0),
 ('opioid_peptid', 0.0),
 ('littl_known', 0.0),
 ('adult_suicid', 0.0),
 ('purpos_studi', 0.0),
 ('feel_lone', 0.0),
 ('juvenil_isol', 0.0),
 ('studi_examin', 0.0),
 ('age_70', 0.0),
 ('live_independ', 0.0),
 ('hear_impair', 0.0),
 ('mental_health', 0.0),
 ('better_togeth', 0.0),
 ('social_connected', 0.0),
 ('qualiti_life', 0.0),
 ('activ_daili', 0.0),
 ('intern_consist', 0.0),
 ('focu_group', 0.0),
 ('age_place', 0.0),
 ('signific_differ', 0.0),
 ('examin_whether', 0.0),
 ('purpos_purpos', 0.0),
 ('day_centr', 0.0),
 ('repres_sampl', 0.0),
 ('age_55', 0.0),
 ('conceptu_model', 0.0),
 ('independ_live', 0.0),
 ('cognit_function', 0.0),
 ('self-rat_health', 0.0),
 ('physic_mental', 0